Doesn't look identical, but not all parameters are specified (i.e. infectious seed).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator

from summer2 import CompartmentalModel
from summer2.parameters import Parameter

In [ ]:
def build_seir_model(
    model_config
) -> CompartmentalModel:

    # Model characteristics
    compartments = (
        "Susceptible", 
        "Pre-infectious", 
        "Infectious", 
        "Immune"
    )
    model = CompartmentalModel(
        times=(0., model_config["end_time"]),
        compartments=compartments,
        infectious_compartments=["Infectious"],
    )
    model.set_initial_population(
        distribution={
            "Susceptible": model_config["total_population"] - model_config["infectious_seed"],
            "Infectious": model_config["infectious_seed"],
        }
    )
    
    # Transitions
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=Parameter("r0") / Parameter("infectious_period"),
        source="Susceptible",
        dest="Pre-infectious"
    )
    model.add_transition_flow(
        name="progression", 
        fractional_rate=1. / Parameter("latent_period"),
        source="Pre-infectious", 
        dest="Infectious"
    )
    model.add_transition_flow(
        name="recovery", 
        fractional_rate=1. / Parameter("infectious_period"), 
        source="Infectious", 
        dest="Immune"
    )
    
    # Output
    model.request_output_for_flow(
        name="incidence", 
        flow_name="progression"
    )
    return model

In [ ]:
config = {
    "total_population": 1e5,
    "infectious_seed": 1.,  # Not specified in text
    "end_time": 120.,
}

parameters = {
    "r0": 13.,
    "latent_period": 7.,
    "infectious_period": 8.,
}

In [ ]:
measles_model = build_seir_model(config)
measles_model.run(parameters=parameters)

#### Panel (a), left - measles

In [ ]:
fig, (left_ax, right_ax) = plt.subplots(1, 2, figsize=(14, 5))

left_ax.plot(
    measles_model.times, 
    measles_model.get_outputs_df()["Susceptible"] / config["total_population"],
    color="0.5",
    label="Susceptible",
)
left_ax.set_ylim(0., 1.)
left_ax.set_ylabel("Proportion susceptible")
left_ax.set_xlim(0., 120.)
left_ax.legend()
left_ax.set_title("Measles")
left_ax.spines.top.set_visible(False)

twin_ax = left_ax.twinx()
twin_ax.plot(
    measles_model.times,
    measles_model.get_derived_outputs_df()["incidence"], 
    color="0", 
    linestyle="--",
    label="Infectious",
)
twin_ax.set_ylim(0., 8000.)
twin_ax.legend(loc=4)
twin_ax.spines.top.set_visible(False)

image = plt.imread("./figures/fig_4_3a_measles.jpg")
right_ax.imshow(image, aspect="auto")
right_ax.axis("off");

#### Panel (b), left - measles

In [ ]:
fig, (left_ax, right_ax) = plt.subplots(1, 2, figsize=(14, 5))

left_ax.plot(
    measles_model.times, 
    measles_model.get_outputs_df()["Susceptible"] / config["total_population"],
    color="0.5",
    label="Susceptible",
)
reproduction_number = parameters["r0"] * measles_model.get_outputs_df()["Susceptible"] / config["total_population"]
left_ax.plot(
    measles_model.times,
    reproduction_number,
    color="0.",
    label="Rn",
)
left_ax.set_ylim(0., 13.)
left_ax.set_ylabel("Proportion susceptible")
left_ax.set_xlim(0., 120.)
left_ax.set_title("Measles")
left_ax.spines.top.set_visible(False)

twin_ax = left_ax.twinx()
twin_ax.plot(
    measles_model.times, 
    measles_model.get_derived_outputs_df()["incidence"], 
    color="0", 
    linestyle="--",
    label="Infectious",
)
twin_ax.set_ylim(0., 8000.)
twin_ax.spines.top.set_visible(False)

left_ax.legend()
twin_ax.legend(loc=4)

image = plt.imread("./figures/fig_4_3b_measles.jpg")
right_ax.imshow(image, aspect="auto")
right_ax.axis("off");

In [ ]:
parameters = {
    "r0": 2.,
    "infectious_period": 2.,
    "latent_period": 2.,
}
flu_model = build_seir_model(config)
flu_model.run(parameters=parameters)

#### Panel (a), right - influenza

In [ ]:
fig, (left_ax, right_ax) = plt.subplots(1, 2, figsize=(14, 5))

left_ax.plot(
    flu_model.times, 
    flu_model.get_outputs_df()["Susceptible"] / config["total_population"],
    color="0.5",
    label="Susceptible",
)
left_ax.set_ylim(0., 1.)
left_ax.set_ylabel("Proportion susceptible")
left_ax.set_xlim(0., 120.)
left_ax.legend()
left_ax.set_title("Influenza")
left_ax.spines.top.set_visible(False)

twin_ax = left_ax.twinx()
twin_ax.plot(
    flu_model.times, 
    flu_model.get_derived_outputs_df()["incidence"], 
    color="0", 
    linestyle="--",
    label="Incidence",
)
twin_ax.set_ylim(0., 8000.)
twin_ax.legend(loc=4)
twin_ax.spines.top.set_visible(False)

image = plt.imread("./figures/fig_4_3a_flu.jpg")
right_ax.imshow(image, aspect="auto")
right_ax.axis("off");

#### Panel (b), right - influenza

In [ ]:
fig, (left_ax, right_ax) = plt.subplots(1, 2, figsize=(14, 5))

left_ax.plot(
    flu_model.times, 
    flu_model.get_outputs_df()["Susceptible"] / config["total_population"],
    color="0.5",
    label="Susceptible",
)
reproduction_number = parameters["r0"] * flu_model.get_outputs_df()["Susceptible"] / config["total_population"]
left_ax.plot(
    flu_model.times,
    reproduction_number,
    color="0.",
    label="Rn",
)
left_ax.set_ylim(0., 2.)
left_ax.set_ylabel("Proportion susceptible")
left_ax.set_xlim(0., 120.)
left_ax.set_title("Influenza")
left_ax.xaxis.set_minor_locator(AutoMinorLocator(5))
left_ax.yaxis.set_minor_locator(AutoMinorLocator(5))
left_ax.set_yticks(np.linspace(0., 2., 5))
left_ax.spines.top.set_visible(False)

twin_ax = left_ax.twinx()
twin_ax.plot(
    flu_model.times, 
    flu_model.get_derived_outputs_df()["incidence"], 
    color="0", 
    linestyle="--",
    label="Incidence",
)
twin_ax.set_ylim(0., 8000.)
twin_ax.spines.top.set_visible(False)

left_ax.legend()
twin_ax.legend(loc=4)

image = plt.imread("./figures/fig_4_3b_flu.jpg")
right_ax.imshow(image, aspect="auto")
right_ax.axis("off");